Proof of Concept - Manipulate datasets in the project
===

In [1]:
# Library Utilitaire pour la gestion de projet
import PythonLib

from os.path import splitext, basename

## CSV

In [5]:
import pandas as pd

author= "Mark S. Daskin"
topic = "NETWORK-AND-DISCRETE-LOCATION"

dataset="49-nodes.csv"

dataset_file = "./Datasets/" + author + "/" + dataset

data = pd.read_csv(dataset_file)

In [6]:
data.head()

,ID,LATITUDE,LONGITUDE,STATE POP,CITY POP,FIXED COST,CITY
0,1,121.467360,38.56685,29760021,369365,115800,< Sacramento CA >
1,2,73.799017,42.66575,17990455,101082,101800,< Albany NY >
2,3,97.750522,30.30588,16986510,465622,72600,< Austin TX >
3,4,84.281399,30.45700,12937926,124773,72400,< Tallahassee FL >
4,5,76.884503,40.27605,11881643,52376,38400,< Harrisburg PA >


## Mongodb

### Create problem

In [7]:
# Generate a connection to our database
import pymongo
from pymongo import MongoClient

mongo_host='ns396089.ip-37-59-38.eu'
mongo_port=32771

client = MongoClient(mongo_host, mongo_port)

# Collection where we wich to store our problem
problems = client[topic][author] # client.[db name].[collection name]

# Prepare the datas to be saved
data.index = data.index.map(str)
record = {
    "metadata": {
        "author": author,
        "topic": topic,
        "dataset": splitext(basename(dataset))[0]
    },
    "data": data.to_dict()
}

# Insert a new entry to the database
problem_id = problems.insert_one(record).inserted_id

### Read problem

In [8]:
# fetch all problems, return an iterator
q = {} # query select the desired intances
f = {} # filter | select the desired fields
records = problems.find(q,f)

In [9]:
list(records)

[{'_id': ObjectId('5ba9a5ab27f7f300226e6e41')},
 {'_id': ObjectId('5bac222a27f7f30021f79267')}]

## Create rules for the database

In [10]:
problems.create_index([('metadata.author', pymongo.DESCENDING),
                       ('metadata.topic', pymongo.DESCENDING),
                       ('metadata.dataset', pymongo.DESCENDING)], unique=True)

'metadata.author_-1_metadata.topic_-1_metadata.dataset_-1'